In [ ]:
import geemap
import ee
Map = geemap.Map()
Map

In [ ]:
#研究区
f_shp =r'E:\Work\boundary\boundary2.shp' #添加面shp（文件名不能有“_”和中文）
shp_ee = geemap.shp_to_ee(f_shp)
Map.addLayer(shp_ee, {}, 'GX')

In [ ]:
def maskL8sr(image):
    # Bit 0 - Fill
    # Bit 1 - Dilated Cloud
    # Bit 2 - Cirrus
    # Bit 3 - Cloud
    # Bit 4 - Cloud Shadow
    qaMask = image.select('QA_PIXEL').bitwiseAnd(int('11111', 2)).eq(0)
    saturationMask = image.select('QA_RADSAT').eq(0)

    # Apply the scaling factors to the appropriate bands.
    opticalBands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
    thermalBands = image.select('ST_B.*').multiply(0.00341802).add(149.0)

    # Replace the original bands with the scaled ones and apply the masks.
    return image.addBands(opticalBands, None, True) \
        .addBands(thermalBands, None, True) \
        .updateMask(qaMask) \
        .updateMask(saturationMask)\
        .copyProperties(image, ["system:time_start",'system:id'])

In [ ]:
def ndvi_isNull(img):
    ndvi = img.normalizedDifference(['SR_B5','SR_B4']).rename('NDVI')
    Msk = ndvi.lte(1).And(ndvi.gt(0))
    ndvi = ndvi.updateMask(Msk)
    xmin = ndvi.reduceRegion(**{#**以dic形式传递，不加则以元组形式
          'reducer': ee.Reducer.min(),
          'geometry': shp_ee,
          'scale': 30,
          'maxPixels': 1e13
        }).get('NDVI')
    xmax = ndvi.reduceRegion(**{
          'reducer': ee.Reducer.max(),
          'geometry': shp_ee,
          'scale': 30,
          'maxPixels': 1e13
        }).get('NDVI')

#     xnum = ee.Number(xmax).subtract(ee.Number(xmin)
    return img.addBands(ndvi).updateMask(Msk).set({"xmin":xmin, "xmax":xmax}).copyProperties(img, ["system:time_start",'system:id'])
def vfc(img): 
    fv = img.expression('(ndvi - xmin) / (xmax - xmin)',{
        'ndvi': img.select("NDVI"),
        'xmin': ee.Number(img.get("xmin")), # ee.Image(xmin)
        'xmax': ee.Number(img.get("xmax"))
        }).rename("fv").toFloat()
#     fv = ndvi.subtract(xmin).divide(xmax.subtract(xmin)).rename('vfc')
#     Msk = img.select("NDVI").lte(1).And(img.select("NDVI").gt(0))
    return img.addBands(fv).copyProperties(img, ["system:time_start",'system:id'])
#     return img.addBands(fv).updateMask(Msk).copyProperties(img, ["system:time_start",'system:id'])


In [ ]:
collection = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2") \
.filter(ee.Filter.calendarRange(2018,2021,'year'))\
.filter(ee.Filter.calendarRange(5,9,'month'))\
.filterBounds(shp_ee) \
.map(maskL8sr)

In [ ]:
print(collection.size().getInfo())

In [ ]:
col_isNull = collection.map(ndvi_isNull)
col_vfc = col_isNull.filter(ee.Filter.notNull(["xmin", "xmax"]))
col_vfc1 = col_vfc.filter(ee.Filter.notEquals("xmax", "xmin"))
col_vfc2 = col_vfc1.map(vfc)

In [ ]:
Map

In [ ]:
Max = col_vfc2.aggregate_array('xmin').getInfo()
# Min = col_vfc2.aggregate_array('xmin').getInfo()
# ID = col_vfc2.aggregate_array('system:id').getInfo()
print(Max)
# print(ID)

In [ ]:
Map.addLayer(col_vfc2.select('fv').mean(), {}, 'fv')

In [ ]:
col_vfc3 = col_vfc2.mean()

In [ ]:
xmin = col_vfc3.select('fv').reduceRegion(**{#**以dic形式传递，不加则以元组形式
          'reducer': ee.Reducer.min(),
          'geometry': shp_ee,
          'scale': 30,
          'maxPixels': 1e13
        }).get('fv')
xmax = col_vfc3.select('fv').reduceRegion(**{
      'reducer': ee.Reducer.max(),
      'geometry': shp_ee,
      'scale': 30,
      'maxPixels': 1e13
    }).get('fv')

In [ ]:
xmin.getInfo()

In [ ]:
xmax.getInfo()

In [ ]:
print(col_vfc2.size().getInfo())

In [ ]:
Map.addLayer(col_vfc3,{}, 'img')

In [ ]:
Map

In [ ]:
for name in names:
    img = col_vfc2.select(name).mean()
    Map.addLayer(img, {}, name)

In [ ]:
col_vfc2.first().bandNames().getInfo()

In [ ]:
names = image.bandNames().getInfo()

In [ ]:
Map.addLayer(image,{}, 'col3')

In [ ]:
Map

In [ ]:
col_vfc2.propertyNames().getInfo()